<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/notebooks/label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 16.4 MB/s 
     |████████████████████████████████| 182 kB 42.4 MB/s 


In [2]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.8 MB/s 
     |████████████████████████████████| 212 kB 52.9 MB/s 
     |████████████████████████████████| 115 kB 58.4 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from transformers import pipeline
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

In [4]:
classifier = pipeline("zero-shot-classification", device = 0, model = 'valhalla/distilbart-mnli-12-1')

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [5]:
dataset = 'cnn_dailymail'
version = '3.0.0'

In [6]:
from datasets import load_dataset

dataset = load_dataset(dataset, version)

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Playing with subsets

In [7]:
# dataset_subset = dataset['test'].shard(num_shards=100, index=0)

In [8]:
# output = classifier(dataset_subset['article'], candidate_labels)

In [9]:
# subset_indices = []

In [10]:
# for i, article in enumerate(output):
#   if article['labels'][0] == 'about sports' and article['scores'][0]>0.6:
#     subset_indices.append(i)

In [11]:
# subset_indices

In [12]:
# a = dataset_subset.select(subset_indices)

In [13]:
# a.save_to_disk('sports_test.hf')

In [14]:
# output[0].keys()

# Actual code

In [15]:
def classify_and_subset(dataset, column, classifier, label, threshold, filename):
  '''
  classifies a text into being about the label or not about the label. 
  text is contained in specified column of dataset. 
  threshold is level of certainty required (0-1)
  returns a subsetted dataset
  '''
  subset_indices = []
  # Zero-shot classification requires all the possible labels listed out. 
  labels = [f'about {label}', f'not about {label}']
  for i, article in tqdm(enumerate(classifier(KeyDataset(dataset, column), labels))):
    if article['labels'][0] == labels[0] and article['scores'][0]>threshold:
      subset_indices.append(i)
  subset = dataset.select(subset_indices)
  subset.save_to_disk(filename)

In [16]:
# subset = classify_and_subset(dataset_subset, 'article', classifier, 'sports', 0.6)

In [17]:
classify_and_subset(dataset['validation'], 'article', classifier, 'sports', 0.6, 'validation_subset')




0it [00:00, ?it/s]


1it [00:12, 12.75s/it]


2it [00:26, 13.36s/it]


3it [00:33, 10.36s/it]


4it [00:38,  8.41s/it]


5it [00:45,  7.96s/it]


6it [00:59,  9.76s/it]


7it [01:07,  9.20s/it]


8it [01:09,  6.95s/it]


9it [01:16,  6.97s/it]


10it [01:22,  6.71s/it]


11it [01:25,  5.64s/it]


12it [01:31,  5.75s/it]


13it [01:35,  5.20s/it]


14it [01:39,  4.66s/it]


15it [01:42,  4.20s/it]


16it [01:46,  4.19s/it]


17it [01:49,  3.74s/it]


18it [01:54,  4.33s/it]


19it [02:06,  6.67s/it]


20it [02:16,  7.70s/it]


21it [02:19,  6.04s/it]


22it [02:20,  4.69s/it]


23it [02:23,  4.23s/it]


24it [02:31,  5.41s/it]


25it [02:40,  6.22s/it]


26it [02:44,  5.78s/it]


27it [02:48,  5.28s/it]


28it [02:51,  4.39s/it]


29it [02:53,  3.77s/it]


30it [02:58,  4.16s/it]


31it [03:11,  6.71s/it]


32it [03:15,  6.04s/it]


33it [03:18,  4.94s/it]


34it [03:31,  7.55s/it]


35it [03:43,  6.38s/it]


KeyboardInterrupt: ignored

In [ ]:
# validation_data_new.save_to_disk('cnn_test_sports.hf')

In [ ]:
# from datasets import load_from_disk
# dataset2 = load_from_disk('cnn_test_sports.hf')

Trying cloud - decided against it due to credential issues from colab

In [ ]:
# bucket = client.bucket('news_data_subset')

In [ ]:
# blob=bucket.blob('sports_test.hf/dataset.arrow')

In [ ]:
# blob.upload_from_filename('sports_test.hf/dataset.arrow')

In [ ]:
# from google.cloud import storage

In [ ]:
# def save_to_cloud(dataset, bucket_name, machine_filename, cloud_filename):
#   # dataset.save_to_disk(machine_filename)
#   client = storage.Client()
#   bucket = client.bucket(bucket_name)
#   for i, filename in enumerate(images.keys()):
#     blob = bucket.blob(cloud_filename)
#     blob.upload_from_filename(machine_filename)
#     # # Delete local data after uploading to cloud
#     # if os.path.exists(machine_filename):
#     #     os.remove(machine_filename)

In [ ]:
# save_to_cloud('news_data_subset', 'sports_test.hf', 'sports_test.hf')

In [ ]:
# save_to_cloud('news_data_subset', 'cnn_test_sports.hf', 'cnn_test_sports.hf')